# Minimum Complementary Energy

Over the past two weeks, we have explored the displacement resulting from distributed pressure and the Hertz contact solution. Now, we shift our focus to real contact problems, emphasizing **Variational Principles** and the **Minimum Complementary Energy** approach.

![Contact model discussed in first step](figures/Problem_mapping.png)

Here we simplify our model as the contact between a rigid rough surface and an elastic flat surface[4].

![rigid rough surface elastic contact](figures/rigid_rough_surface_elastic_contact.jpg)


We want to know pressure distribution $P(x,y)$, and we define our gap function as:

$$
g(x, y)=u_z(x, y)-h(x, y)
$$

where $u_z(x, y)$ is the normal displacement, the separation of two surfaces $h(x,y)$ can sometimes be $h(r)=-\frac{r^2}{2 R}$, like Rey(2017)[5].

As the first chapiter of Lucas(2020)[1], the potential energy can be derived from elastic energy relationship $\frac{1}{2} \int_v \varepsilon: \mathbb{C}: \varepsilon d v$ to:

$$
E_p\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s
$$

If we minimize this potential energy with respect to $u_z$, then we call it **Primal minimization problem**, we will end up with a displacement solution.

$$
\min _{g\left[u_z\right] \geqslant 0} E_p\left(u_z, p\left[u_z\right]\right)
$$

Here we focus on complementary energy minimization, called **Dual minimization problem**, where complementary energy is defined as:

$$
E_c\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s-\int_s p h d s
$$

Then the minimization could be:

$$
\min _{p \geqslant 0} E_c\left(u_z[p], p\right)
$$

It is obvious that we will end up with a pressure solution in this procedure.



## Q1: Legendre transform for dual optimization?

From what we discuss above, we have two constraints, 

$$
\frac{1}{S} \int_s p d s=\bar{p}
$$

And

$$
p \geq 0
$$

To minimize the complementary energy, we consider:

$$
\lim _{\varepsilon \rightarrow 0} \frac{E_c(p+\varepsilon v)-E_c(p)}{\varepsilon}
$$

Here we can derive a gap function by:

$$
\frac{\partial E_c}{\partial p}=u_z[p]-h=g
$$



We choose the same background as a start point to verify this energy minimization, and define $ \bar{p}=\frac{W}{S} $

### *$u_z$* is our unknown in this case, but we know the relationship between *$u_z$* and pressure *$P$*, we will then treat *$u_z$* as *$u_z(P)$*. The Hertz solution is derived from elastic theory, here we want to derive from energy theory, finally we can compare with Hertz solution.

#  Complementary energy minimization for Normal contact

We follow the steps of Lucas(2020)[1], and finally we will have two alrgorithms to implement, Steepest descent[6] and Constrained conjugate gradient[7]. First we give the notation as reference:

Let the open $\mathcal{B} \subset \mathbb{R}^3$ be a deformable elastic solid of boundary $\partial \mathcal{B}$ with outward normal $\boldsymbol{n}$. Let $u$ be the displacement vector field of $\mathcal{B}$.

We have the following strain-stress relationship in the context of continuum mechanics and elasticity theory:

$$
\boldsymbol{\varepsilon}[\boldsymbol{u}]:=\frac{1}{2}\left(\boldsymbol{\nabla} \boldsymbol{u}+\boldsymbol{\nabla} \boldsymbol{u}^T\right)
$$
$$
\sigma[u]:=C: \varepsilon[u]
$$

We define the traction vector of the displacement field $\boldsymbol{u}$ as:

$$
T[u]:=\left.\sigma[u]\right|_{\partial \mathcal{B}} \cdot \boldsymbol{n}
$$

$\boldsymbol{b}$ is given as the volumetric force density, then with given boundary condition, we can express the principle of minimum potential energy:

$$
\inf _{\boldsymbol{u} \in \mathrm{KA}(\mathcal{B})}\left\{I(\boldsymbol{u})=\frac{1}{2} \int_{\mathcal{B}} \boldsymbol{\sigma}[\boldsymbol{u}]: \boldsymbol{\varepsilon}[\boldsymbol{u}] \mathrm{d} V-\int_{\mathcal{B}} \boldsymbol{b} \cdot \boldsymbol{u} \mathrm{d} V-\int_{\partial \mathcal{B}} T[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S\right\}
$$

which involves finding $u \in \mathrm{KA}(\mathcal{B})$, i.e. a kinematically admissible displacement field, that minimizes the total potential energy of the system.






As we mentioned before, we want to minimize complementary energy for a non-friction, non-adhesion contact between the boundary $\partial \mathcal{B}$ and a surface $\mathcal{S}$, we can introduce the gap function as:

$$
g[\boldsymbol{u}]:=\left.\boldsymbol{u} \cdot \boldsymbol{e}_3\right|_{\partial \mathcal{B}}-h
$$

The boundary conditions on $\partial \mathscr{B}$ are hence expressed with the Hertz-Signorini-Moreau conditions:

$$
\begin{aligned}
g[\boldsymbol{u}] & \geq 0 \\
p[\boldsymbol{u}]:=\boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{e}_3 & \geq 0 \\
g[\boldsymbol{u}] p[\boldsymbol{u}] & =0
\end{aligned}
$$

The contact condition that we want to apply for **Steepest descent** algorithm as stopping criteria is:

$$
g[\boldsymbol{u}] p[\boldsymbol{u}]=0
$$



### Variational form

For this contact problem, we have two constraints, 

$$
\frac{1}{S} \int_s p d s=\bar{p}
$$

And

$$
p \geq 0
$$

To introduce the contact constraints, one needs to define the space of kinematically admissible displacement fields $\mathrm{KA}(\mathcal{B})$. The first consideration defines the function space

$$
\bar{H}^1\left(\mathcal{B}_p ; \mathbb{R}^3\right)=\left\{\boldsymbol{u}+\overline{\boldsymbol{u}} \mid \boldsymbol{u} \in H^1\left(\mathcal{B}_p ; \mathbb{R}^3\right), \overline{\boldsymbol{u}} \in \mathbb{R}^3 \text { with } \widehat{\mathbf{u}}(\mathbf{0}, \bullet)=0\right\}
$$

which is a space of $\mathcal{B}_p$-periodic functions whose fundamental mode (i.e. the horizontal average) is constant with respect to $x_3$. The space of admissible solutions to the contact problem is:

$$
\mathrm{KA}\left(\mathcal{B}_p\right):=\left\{\boldsymbol{u} \in \bar{H}^1\left(\mathcal{B}_p ; \mathbb{R}^3\right): g[\boldsymbol{u}] \geq 0\right\}
$$

The minimization principle is then written as:

$$
\inf _{\boldsymbol{u} \in \operatorname{KA}\left(\mathcal{B}_p\right)}\left\{\frac{1}{2} \int_{\mathcal{B}_p} \boldsymbol{\sigma}[\boldsymbol{u}]: \boldsymbol{\varepsilon}[\boldsymbol{u}] \mathrm{d} V\right\}
$$

which is a quadratic program with unilateral constraints. Kalker(1977)[3] has shown that the above problem can be expressed only in terms of boundary integrals, provided $\boldsymbol{u}$ additionally satisfies the local equilibrium(strong Euler-Lagrange equation) $\operatorname{div}(\sigma[\boldsymbol{u}])+\boldsymbol{b}=\mathbf{0} \quad \text { a.e. in } \mathcal{B}$:

$$
\begin{aligned}
\frac{1}{2} \int_{\mathcal{B}_p} \sigma[\boldsymbol{u}]: \boldsymbol{\varepsilon}[\boldsymbol{u}] \mathrm{d} V & =\frac{1}{2} \int_{\mathcal{B}_p} \boldsymbol{\sigma}[\boldsymbol{u}]: \boldsymbol{\nabla} \boldsymbol{u} \mathrm{d} V \\
& =\frac{1}{2} \int_{\mathcal{B}_p}(\operatorname{div}(\sigma[\boldsymbol{u}] \cdot \boldsymbol{u})-\operatorname{div}(\sigma[\boldsymbol{u}]) \cdot \boldsymbol{u}) \mathrm{d} V \\
& =\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{u} \cdot \boldsymbol{\sigma}[\boldsymbol{u}] \cdot \boldsymbol{n} \mathrm{d} V
\end{aligned}
$$

so that the contact problem is equivalent to

$$
\inf _{\boldsymbol{u} \in \operatorname{KA}\left(\mathcal{B}_p\right)}\left\{I_u(\boldsymbol{u})=\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S\right\}
$$

This minimization problem can now be solved by quadratic programming (see Rey(2017)[5]), but optimization techniques are usually applied to the dual of $\inf _{\boldsymbol{u} \in \operatorname{KA}\left(\mathcal{B}_p\right)}\left\{I_u(\boldsymbol{u})=\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S\right\}$, which we derive by associating a Lagrange multiplier $\lambda$ to the non-penetration constraint $g[\boldsymbol{u}] \geq 0$.  The Lagrangian of the problem is:

## Question: quadratic programming, dual problem?

$$
\mathcal{L}(\boldsymbol{u}, \lambda)=\frac{1}{2} \int_{\partial \mathcal{B}_p} \boldsymbol{T}[\boldsymbol{u}] \cdot \boldsymbol{u} \mathrm{d} S-\int_{\partial \mathcal{B}_p} \lambda\left(\boldsymbol{u} \cdot \boldsymbol{e}_3-h\right) \mathrm{d} S
$$

The Karush-Kuhn-Tucker optimality conditions imply stationarity of $\mathcal{L}$ with respect to $\boldsymbol{u}$ so that we can express $\lambda$ :

$$
\begin{aligned}
\frac{\partial \mathcal{L}}{\partial \boldsymbol{u}}=\mathbf{0} & \Leftrightarrow T[u]-\lambda \boldsymbol{e}_3=0 \\
& \Leftrightarrow \lambda=T[\boldsymbol{u}] \cdot \boldsymbol{e}_3 \\
& \Leftrightarrow \boldsymbol{u}=\mathcal{M}\left[\lambda \boldsymbol{e}_3\right] 
\end{aligned}
$$

$\lambda$ can be physically interpreted as the normal tractions acting on the surface. Replacing $\boldsymbol{u}=\mathcal{M}\left[p \boldsymbol{e}_3\right]$ in $\mathcal{L}$ (effectively computing the Legendre transform of $I_u$ ) leads to the quadratic program

$$
\inf _{p \boldsymbol{e}_3 \in \mathrm{SA}\left(\mathcal{B}_p\right)}\left\{I_p(p)=\frac{1}{2} \int_{\partial \mathcal{B}_p} p \boldsymbol{e}_3 \cdot \mathcal{M}\left[p \boldsymbol{e}_3\right] \mathrm{d} S-\int_{\partial \mathcal{B}_p} p h \mathrm{~d} S\right\}
$$

where the primary unknown $p$ is the normal surface traction and SA is the space of statically admissible tractions, i.e. tractions that satisfy $p \geq 0$ (for non-adhesive contact) and $\int_{\partial \mathcal{B}} p \mathrm{~d} S=$ $W$.

### Just not to be confused, $W$ is normal total load and we define $ \bar{p}=\frac{W}{S} $. 

In [1]:
import numpy as np
from scipy.optimize import minimize

#define the parameters
W = 1e2  # total load
R = 1  # radius of demi-sphere
L = 2  # domain size
S = L**2  # domain area



#define material parameters
E = 1e3  # Young's modulus
nu = 0.3  # Poisson's ratio
E_star = E / (1 - nu**2)  # plane strain modulus 

#We generate a 2D coordinate space
n = 100
m = 100

x, y = np.meshgrid(np.linspace(0, L, n, endpoint=False), np.linspace(0, L, m, endpoint=False))#notice here that we use endpoint=False to avoid having the last point

x0 = 1
y0 = 1


## the separation function h depends on the contact model, since we just use a simple semi-sphere/flat surface contact, we keep it as $h(r)=-\frac{r^2}{2 R}$

In [2]:
# Define the separation h          #Since we are using the same model, we can use the same h function
h = - (x**2 + y**2)/(2*R)

## Minimization with FFT

First we use **scipy** library functions, *scipy.optimize.minimize* and *scipy.optimize.LinearConstraint* and FFT method with the nice property for convolution integral. We have the following convolution product:

$$
\mathcal{F}(u)= \mathcal{F}(f) \bullet \mathcal{F}(p) 
$$

In [3]:

p_bar = W / S  

# ?????
def pressure_discritization(x, y, x0, y0):
    return p_bar


p_initial = pressure_discritization(x, y, x0, y0)# initial pressure field

p0_fourier = np.fft.fft2(p_initial, norm='ortho')  # Fourier transform of the uniform pressure


IndexError: cannot do a non-empty take from an empty axes.

We also need to define our Kernel function in fourrier domain:

$$
\mathcal{F}(f)=\frac{2}{E^* \sqrt{q_x^2+q_y^2}}
$$


In [ ]:
#we define the frequency with q_x and q_y
q_x = 2 * np.pi * np.fft.fftfreq(n, d=L/n)
q_y = 2 * np.pi * np.fft.fftfreq(m, d=L/m)
QX, QY = np.meshgrid(q_x, q_y)

kernel_fourier = np.zeros_like(QX)  # Initialize the kernel array
#non_zero_indices = (QX**2 + QY**2) != 0  # Avoid division by zero
kernel_fourier = 2 / (E_star * np.sqrt(QX**2 + QY**2))
kernel_fourier[0, 0] = 0  # Set the zero frequency component to zero

## Possible to have negative value, because we set the *average value $C_0$* to be zero.

In [ ]:
#Calculate the displacement field in fourrier space
displacement_fourrier = p0_fourier * kernel_fourier

#We calculate the displacement field in real space
displacement_real = np.fft.ifft2(displacement_fourrier, norm='ortho')


In [ ]:


# define the complementary energy as cost function
def cost_function(displacement_real, h, P):
    return np.sum(displacement_real*P)/2 - np.sum(h*P)

# define the gradient of the cost function as jacobian
def gradient(displacement_real, h):
    return displacement_real-h

jac = gradient(displacement_real, h)

 # define the length of the pressure vector
n = len(p_bar)  # p_bar is initial guess for the pressure
# first non_negative_constraint for the pressure
non_negative_constraint = minimize.LinearConstraint(np.eye(n), np.zeros(n), np.inf*np.ones(n))

# second avarage_constraint for the pressure
average_pressure_constraint = minimize.LinearConstraint(np.ones((1, n))/S, [p_bar*S], [p_bar*S])


result = minimize(cost_function, p_bar, method='trust-constr', jac=jac, constraints=[non_negative_constraint, average_pressure_constraint])

#print(result)


TypeError: object of type 'float' has no len()

There is one more notation that needs to be clarified, $\mathcal{M}$ is a boundary integral operator defined in terms of fundamental solutions. Consequently, the integral operator $\mathcal{M}$ can be expressed as a convolution with respect to the horizontal coordinates. 

The gradient computation of the functional $I_p$ is straightforward with the use of $\mathcal{M}$:

$$
\nabla I_p(p)=\mathcal{M}\left[p \boldsymbol{e}_3\right] \cdot \boldsymbol{e}_3-h=u_3-h=g
$$


## Steepest descent algorithm

The approach proposed by Stanley and Kato(1997)[6] consists in taking steps in the $-\nabla I_p=-g$ direction then shifting and truncating the normal traction so that both the unilateral and equilibrium constraints are satisfied. Besides, its convergence rate is sub-optimal.



### Algorithm 1: Stanley and Kato (1997) steepest descent algorithm.

**Inputs:** normal total load  $W$, surface profile $\mathbf{H}$, tolerance  $\varepsilon$, maximum number of iterations  $N_{\text{max}}$.

1. Initialize $ P $ with the average constant load initial guess:
   $$ P \leftarrow \frac{W}{|\partial \mathbf{B}P|} $$
2. Set $ h_{\text{norm}} $ to the norm of  $\mathbf{H}$ :
   $$ h_{\text{norm}} \leftarrow \|\mathbf{H}\| $$
3. Set iteration counter $ k $ to zero:
   $$ k \leftarrow 0 $$

**Repeat until** $ e < \varepsilon $ **or** $ k > N_{\text{max}} $:

4. Calculate the gradient $ G $:
   $$ G \leftarrow M[\mathbf{P}e_3] \cdot e_3 - \mathbf{H} $$
5. Update $ P $ by subtracting $ G $:
   $$ P \leftarrow P - G $$
6. Find $ \alpha_0 $ that solves the following equation (where $ (\bullet)_+ $ is the ramp function):
   $$ \text{Find } \alpha_0 \text{ solution of } \Sigma (P + \alpha_0)_+ - W = 0 $$
7. Update $ P $ with $ \alpha_0 $:
   $$ P \leftarrow (P + \alpha_0)_+ $$
8. Calculate the error on complementarity $ e $:
   $$ e \leftarrow |P \cdot (G - \min(G))|/(Wh_{\text{norm}}) $$
9. Increment the iteration counter $ k $:
   $$ k \leftarrow k + 1 $$

**After exiting the loop**:

10. Ensure a positive gap by updating \( G \):
    $$ G \leftarrow G - \min(G) $$


In [ ]:
h_norm = np.abs(h(x, y, R)) #norm of the h function

alpha_0 = 1e-2  # learning rate
tol = 1e-6  # tolerance for convergence
iter_max = 1000  # maximum number of iterations

# Initialize P with the average constant load initial guess
P = np.full((n, m), p_bar)


P = p_bar #Initial guess for the pressure

for k in range(0, iter_max): 
    if np.abs(error) < tol:

        #gradient(gap)
        G = 
        P = P - G

        '''
        # define the length of the pressure vector
        n = len(p_bar)  # p_bar is initial guess for the pressure
        # first non_negative_constraint for the pressure
        non_negative_constraint = minimize.LinearConstraint(np.eye(n), np.zeros(n), np.inf*np.ones(n))

        # second avarage_constraint for the pressure
        average_pressure_constraint = minimize.LinearConstraint(np.ones((1, n))/S, [p_bar*S], [p_bar*S])
        '''


        p = 

        error =



        k += 1
        if np.abs(error) < tol:
            p_bar = p
        else:
            print('Convergence failed')

    G = G - min(0, G)  # project the gradient onto the non-negative orthant
            
    else:
        print('Convergence failed')
        break




we give our analytical normal displacement solution by Hertz

$$
\bar{u}_z=\frac{1-\nu^2}{E} \frac{\pi p_0}{4 a}\left(2 a^2-r^2\right), \quad r \leqslant a
$$


## Comparasion with other online solver

## Reference:

[1] Frérot, Lucas Henri Galilée. ‘Bridging Scales in Wear Modeling with Volume Integral Methods for Elastic-Plastic Contact’, n.d.

[2] Yastrebov, Vladislav A. Numerical Methods in Contact Mechanics. Numerical Methods in Engineering Series. Hoboken, NJ: Wiley, 2013.

[3] Kalker, J. J. ‘Variational Principles of Contact Elastostatics’. IMA Journal of Applied Mathematics 20, no. 2 (1977): 199–219. https://doi.org/10.1093/imamat/20.2.199.

[4] Vladislav A. Yastrebov, Contact mechanics and elements of tribology, Lecture 4.b, Contact and transport at small scales, Open Course Contact Mechanics and Elements of Tribology, January 23, 2024, https://cmet.yastrebov.fr/index.html

[5] Rey, Valentine, Guillaume Anciaux, and Jean-François Molinari. ‘Normal Adhesive Contact on Rough Surfaces: Efficient Algorithm for FFT-Based BEM Resolution’. Computational Mechanics 60, no. 1 (July 2017): 69–81. https://doi.org/10.1007/s00466-017-1392-5.

[6] stanly 1997

[7] Polonsky and Keer (1999b) 

[8] https://gitlab.com/tamaas/tamaas/-/blob/master/examples/scipy_penalty.py?ref_type=heads

[9] https://www.pecms.cn/